In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout
from keras.preprocessing import image

import warnings
warnings.filterwarnings('ignore')

In [5]:
# pip install seaborn

In [9]:
train_image='D:/Documents/Capstone/Project -2/Dataset/Full/train'

In [10]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 zoom_range=0.2,
                                 shear_range=0.1,
                                vertical_flip=False)

In [11]:
training_set=train_datagen.flow_from_directory(train_image,
                                              target_size=(224,224),
                                              class_mode='categorical',
                                              batch_size=5)


Found 1852 images belonging to 11 classes.


In [13]:
test_datagen = ImageDataGenerator(rescale = 1./255,
                                 zoom_range=0.2)

test_set=train_datagen.flow_from_directory('D:/Documents/Capstone/Project -2/Dataset/Full/test',
                                              target_size=(224,224),
                                              class_mode='categorical',
                                              batch_size=5)

Found 910 images belonging to 11 classes.


In [15]:
base_model = tf.keras.applications.InceptionResNetV2(
                     include_top=False,
                     weights='imagenet',
                     input_shape=(224,224,3)
                     )
 
base_model.trainable=False
 
model = tf.keras.Sequential([
        base_model,  
        tf.keras.layers.BatchNormalization(renorm=True),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(11, activation='softmax')
    ])

219070464/219055592 [==============================] - 7382s 34us/step


In [16]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# categorical cross entropy is taken since its used as a loss function for
# multi-class classification problems where there are two or more output labels.
# using Adam optimizer for better performance
# other optimizers such as sgd can also be used depending upon the model

In [17]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 5, 5, 1536)       54336736  
 onal)                                                           
                                                                 
 batch_normalization_203 (Ba  (None, 5, 5, 1536)       10752     
 tchNormalization)                                               
                                                                 
 global_average_pooling2d (G  (None, 1536)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               786944    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                        

In [18]:
early = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=0.001, restore_best_weights=True)
# early stopping call back

In [19]:
batch_size=32
STEP_SIZE_TRAIN = training_set.n//training_set.batch_size
STEP_SIZE_VALID = test_set.n//test_set.batch_size

# fit model
history = model.fit(training_set, steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=test_set,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10,
                    callbacks=[early])

Epoch 1/10
185/185 [==============================] - 617s 3s/step - loss: 1.6290 - accuracy: 0.4663 - val_loss: 1.5336 - val_accuracy: 0.5198
Epoch 2/10
185/185 [==============================] - 559s 3s/step - loss: 1.2503 - accuracy: 0.5874 - val_loss: 1.1369 - val_accuracy: 0.6407
Epoch 3/10
185/185 [==============================] - 522s 3s/step - loss: 1.0394 - accuracy: 0.6710 - val_loss: 1.1522 - val_accuracy: 0.6308
Epoch 4/10
185/185 [==============================] - 543s 3s/step - loss: 0.9083 - accuracy: 0.7009 - val_loss: 1.0079 - val_accuracy: 0.6835
Epoch 5/10
185/185 [==============================] - 530s 3s/step - loss: 0.8222 - accuracy: 0.7264 - val_loss: 0.9971 - val_accuracy: 0.7110
Epoch 6/10
185/185 [==============================] - 524s 3s/step - loss: 0.7198 - accuracy: 0.7628 - val_loss: 0.8999 - val_accuracy: 0.7319
Epoch 7/10
185/185 [==============================] - 537s 3s/step - loss: 0.6524 - accuracy: 0.7818 - val_loss: 0.8015 - val_accuracy: 0.7637